# Feature Creation - V2

The goal of this stage is to achieve the following 3 things:

1. Clean the data frame we created in fc_1

2. Lag the features we created in fc_1 
    
    - We calculated a given favorited teams ppg for a given week. However this calculates the ppg for the favorited team **through** a given week.
    
    - Our model needs to predict a given outcome based on a favorite/underdogs stats **up to** a given week
    
    - For example, if we want to predict whether the Cheifs will win in week 3 of a given NFL season, we want to know their ppg up to that week (what was there ppg after week 2). We do not want to take into account there ppg through a given week.
    

3. Impute data for the Nan values due to the lagging of certain variables

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('/Users/epainter/Desktop/bet_model_v2/data/processing/fc_1.csv')

In [4]:
df.head()

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,C_fav,CB_fav,DI_fav,Edge_fav,LB_fav,LG_fav,LT_fav,QB_fav,RB_fav,RG_fav,RT_fav,S_fav,TE_fav,WR_fav,C_und,CB_und,DI_und,Edge_und,LB_und,LG_und,LT_und,QB_und,RB_und,RG_und,RT_und,S_und,TE_und,WR_und,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,team_division_fav,team_division_und,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,66.0,64.27,73.70,55.75,59.30,67.4,71.2,90.2,75.35,59.7,89.3,73.45,86.1,74.73,67.3,67.30,65.53,75.35,67.20,59.1,75.1,82.4,74.05,53.4,59.4,69.60,56.60,71.07,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,AFC West,AFC South,34.0,20.0,0.0,1.0,369.0,360.0,75.55,66.49,360.0,369.0,1.0,0.0,20.0,34.0,65.29,69.00,0,1.0,0.0,1.0,0.0,14.0,-14.0,1.0,0.0,0.0,0.0
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,72.1,60.30,79.53,69.50,69.60,87.7,79.7,75.8,62.95,66.6,53.0,65.35,73.1,77.27,56.9,79.33,65.10,59.60,75.60,60.3,74.1,92.0,77.90,79.3,63.6,68.10,73.25,73.63,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,NFC South,NFC West,25.0,38.0,2.0,0.0,506.0,383.0,72.02,72.33,383.0,506.0,0.0,2.0,38.0,25.0,68.86,69.55,0,0.0,1.0,0.0,1.0,-13.0,13.0,0.0,0.0,0.0,1.0
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,68.7,71.93,74.53,66.85,78.60,63.8,89.4,90.1,81.25,63.6,73.6,73.35,88.5,63.37,72.0,65.47,71.60,83.45,59.67,74.2,88.7,73.5,82.70,56.7,80.5,68.10,72.30,73.45,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,AFC North,AFC North,38.0,6.0,1.0,3.0,381.0,306.0,75.81,74.89,306.0,381.0,3.0,1.0,6.0,38.0,73.05,69.66,1,1.0,0.0,1.0,0.0,32.0,-32.0,1.0,0.0,0.0,0.0
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,67.3,60.20,61.85,68.50,57.03,55.8,73.1,64.1,80.30,63.1,51.8,75.85,60.2,75.43,72.0,69.73,68.17,70.00,73.75,56.1,83.0,63.2,73.80,65.6,60.6,81.25,68.15,75.90,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,AFC East,AFC East,27.0,17.0,2.0,2.0,404.0,254.0,65.68,68.71,254.0,404.0,2.0,2.0,17.0,27.0,64.69,72.58,1,1.0,0.0,1.0,0.0,10.0,-10.0,1.0,0.0,0.0,0.0
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,71.0,59.33,68.45,63.85,65.70,74.0,64.9,79.9,80.05,61.8,69.1,73.60,83.2,72.33,63.4,59.70,87.05,74.20,58.37,58.6,79.3,73.0,80.60,63.6,76.2,77.53,51.80,71.20,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,AFC West,NFC South,34.0,30.0,0.0,0.0,372.0,388.0,72.92,68.63,388.0,372.0,0.0,0.0,30.0,34.0,66.19,71.37,0,1.0,0.0,1.0,0.0,4.0,-4.0,0.0,1.0,0.0,0.0


In [5]:
df.isna().sum()

favorite                 0
underdog                 0
fav_home                 0
fav_score                0
und_score                0
                        ..
und_avg_margin_last_3    0
fav_home_win_pct         0
fav_away_win_pct         0
und_home_win_pct         0
und_away_win_pct         0
Length: 77, dtype: int64

# 1.) Clean Data Frame

I want to first get rid of columns that do not serve any purpose and rename the columns so that they are much easier to change/move/manipulate/etc.

In [6]:

cols_to_drop = ['C_fav', 'CB_fav', 'DI_fav', 'Edge_fav', 'LB_fav', 'LG_fav',
       'LT_fav', 'QB_fav', 'RB_fav', 'RG_fav', 'RT_fav', 'S_fav', 'TE_fav',
       'WR_fav', 'C_und', 'CB_und', 'DI_und', 'Edge_und', 'LB_und', 'LG_und',
       'LT_und', 'QB_und', 'RB_und', 'RG_und', 'RT_und', 'S_und', 'TE_und',
       'WR_und', 'team_division_fav', 'team_division_und']

filtered_df = df.drop(columns=cols_to_drop)

filtered_df


,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_running_avg,und_running_avg,fav_turnover_running_avg,und_turnover_running_avg,fav_yards_running_avg,und_yards_running_avg,off_rating_fav,off_rating_und,fav_def_yards_running_avg,und_def_yards_running_avg,fav_def_turnovers_running_avg,und_def_turnovers_running_avg,fav_def_points_running_avg,und_def_points_running_avg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_margin_last_3,und_avg_margin_last_3,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,34.00,20.00,0.00,1.00,369.00,360.00,75.55,66.49,360.00,369.00,1.00,0.00,20.00,34.00,65.29,69.00,0,1.00,0.00,1.0,0.0,14.00,-14.00,1.00,0.00,0.00,0.00
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,25.00,38.00,2.00,0.00,506.00,383.00,72.02,72.33,383.00,506.00,0.00,2.00,38.00,25.00,68.86,69.55,0,0.00,1.00,0.0,1.0,-13.00,13.00,0.00,0.00,0.00,1.00
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,38.00,6.00,1.00,3.00,381.00,306.00,75.81,74.89,306.00,381.00,3.00,1.00,6.00,38.00,73.05,69.66,1,1.00,0.00,1.0,0.0,32.00,-32.00,1.00,0.00,0.00,0.00
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,27.00,17.00,2.00,2.00,404.00,254.00,65.68,68.71,254.00,404.00,2.00,2.00,17.00,27.00,64.69,72.58,1,1.00,0.00,1.0,0.0,10.00,-10.00,1.00,0.00,0.00,0.00
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,34.00,30.00,0.00,0.00,372.00,388.00,72.92,68.63,388.00,372.00,0.00,0.00,30.00,34.00,66.19,71.37,0,1.00,0.00,1.0,0.0,4.00,-4.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023,Kansas City Chiefs,Los Angeles Chargers,13.0,12.0,268.0,1.0,353.0,1.0,20.35,21.82,1.24,1.65,329.35,351.29,72.14,75.52,362.94,289.76,1.24,1.00,23.41,17.29,62.80,67.67,1,0.29,0.65,0.0,0.6,-3.33,1.00,0.22,0.38,0.50,0.78
1067,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023,New York Giants,Philadelphia Eagles,27.0,10.0,415.0,1.0,299.0,4.0,25.47,15.65,1.65,1.12,354.35,280.00,79.50,71.33,356.12,361.71,1.06,1.82,25.18,23.94,75.67,71.29,1,0.65,0.35,0.2,0.4,-4.33,2.67,0.75,0.56,0.50,0.22
1068,San Francisco 49ers,Los Angeles Rams,1,20,21,-5.5,40.0,0,0.0,1,18,2023,Los Angeles Rams,San Francisco 49ers,21.0,20.0,258.0,1.0,300.0,1.0,28.88,23.76,1.06,1.06,398.41,359.29,72.71,67.38,303.94,337.88,1.65,0.88,17.53,22.18,74.29,65.36,1,0.71,0.59,0.6,0.8,0.67,3.33,0.62,0.78,0.62,0.56
1069,Dallas Cowboys,Washington Commanders,0,38,10,-13.0,47.5,1,1.0,1,18,2023,Dallas Cowboys,Washington Commanders,38.0,10.0,440.0,1.0,180.0,3.0,29.94,19.35,0.94,1.88,371.59,312.76,69.80,64.66,299.71,388.94,1.53,1.06,18.53,30.47,74.82,72.46,1,0.71,0.24,0.6,0.0,9.00,-15.67,1.00,0.44,0.12,0.33


In [7]:
# Chaning column names

filtered_df = filtered_df.rename(columns={'fav_running_avg': 'fav_ppg',
       'und_running_avg': 'und_ppg', 'fav_turnover_running_avg': 'fav_topg',
       'und_turnover_running_avg': 'und_topg', 'fav_yards_running_avg': 'fav_ypg',
       'und_yards_running_avg': 'und_ypg', 'fav_def_yards_running_avg': 'fav_yapg',
                                  'und_def_yards_running_avg': 'und_yapg',
       'fav_def_turnovers_running_avg': 'fav_tofpg', 'und_def_turnovers_running_avg': 'und_tofpg',
       'fav_def_points_running_avg': 'fav_papg', 'und_def_points_running_avg': 'und_papg',
       'fav_avg_margin_last_3': 'fav_avg_mov', 'und_avg_margin_last_3': 'und_avg_mov'})

filtered_df


,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_ppg,und_ppg,fav_topg,und_topg,fav_ypg,und_ypg,off_rating_fav,off_rating_und,fav_yapg,und_yapg,fav_tofpg,und_tofpg,fav_papg,und_papg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_mov,und_avg_mov,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,34.00,20.00,0.00,1.00,369.00,360.00,75.55,66.49,360.00,369.00,1.00,0.00,20.00,34.00,65.29,69.00,0,1.00,0.00,1.0,0.0,14.00,-14.00,1.00,0.00,0.00,0.00
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,25.00,38.00,2.00,0.00,506.00,383.00,72.02,72.33,383.00,506.00,0.00,2.00,38.00,25.00,68.86,69.55,0,0.00,1.00,0.0,1.0,-13.00,13.00,0.00,0.00,0.00,1.00
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,38.00,6.00,1.00,3.00,381.00,306.00,75.81,74.89,306.00,381.00,3.00,1.00,6.00,38.00,73.05,69.66,1,1.00,0.00,1.0,0.0,32.00,-32.00,1.00,0.00,0.00,0.00
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,27.00,17.00,2.00,2.00,404.00,254.00,65.68,68.71,254.00,404.00,2.00,2.00,17.00,27.00,64.69,72.58,1,1.00,0.00,1.0,0.0,10.00,-10.00,1.00,0.00,0.00,0.00
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,34.00,30.00,0.00,0.00,372.00,388.00,72.92,68.63,388.00,372.00,0.00,0.00,30.00,34.00,66.19,71.37,0,1.00,0.00,1.0,0.0,4.00,-4.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023,Kansas City Chiefs,Los Angeles Chargers,13.0,12.0,268.0,1.0,353.0,1.0,20.35,21.82,1.24,1.65,329.35,351.29,72.14,75.52,362.94,289.76,1.24,1.00,23.41,17.29,62.80,67.67,1,0.29,0.65,0.0,0.6,-3.33,1.00,0.22,0.38,0.50,0.78
1067,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023,New York Giants,Philadelphia Eagles,27.0,10.0,415.0,1.0,299.0,4.0,25.47,15.65,1.65,1.12,354.35,280.00,79.50,71.33,356.12,361.71,1.06,1.82,25.18,23.94,75.67,71.29,1,0.65,0.35,0.2,0.4,-4.33,2.67,0.75,0.56,0.50,0.22
1068,San Francisco 49ers,Los Angeles Rams,1,20,21,-5.5,40.0,0,0.0,1,18,2023,Los Angeles Rams,San Francisco 49ers,21.0,20.0,258.0,1.0,300.0,1.0,28.88,23.76,1.06,1.06,398.41,359.29,72.71,67.38,303.94,337.88,1.65,0.88,17.53,22.18,74.29,65.36,1,0.71,0.59,0.6,0.8,0.67,3.33,0.62,0.78,0.62,0.56
1069,Dallas Cowboys,Washington Commanders,0,38,10,-13.0,47.5,1,1.0,1,18,2023,Dallas Cowboys,Washington Commanders,38.0,10.0,440.0,1.0,180.0,3.0,29.94,19.35,0.94,1.88,371.59,312.76,69.80,64.66,299.71,388.94,1.53,1.06,18.53,30.47,74.82,72.46,1,0.71,0.24,0.6,0.0,9.00,-15.67,1.00,0.44,0.12,0.33


# 2.) Lagging Features

Since our data frame is cleaned up, we can now begin to lag our variables.

I am sure there is a much more efficient way to do this, but for me, this is what worked.

In [8]:
# First we will lag the PPG

def process_team_stats(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_score'] = np.where(team_df['favorite'] == team_name, team_df['fav_score'], team_df['und_score'])
    team_df['team_ppg'] = np.where(team_df['favorite'] == team_name, team_df['fav_ppg'], team_df['und_ppg'])
    
    # Create lagged ppg column
    team_df['team_ppg_lag'] = team_df['team_ppg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_ppg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_ppg_lag'], np.nan)
    team_df['und_ppg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_ppg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_score', 'und_score',
                    'fav_ppg_lag', 'und_ppg_lag']]

def all_teams_ppg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_team_stats(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_ppg_lag': 'first',
        'und_ppg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [9]:
# Next we will lag the TOPG

def process_topg_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_to'] = np.where(team_df['favorite'] == team_name, team_df['fav_topg'], team_df['und_topg'])
    team_df['team_topg'] = np.where(team_df['favorite'] == team_name, team_df['fav_topg'], team_df['und_topg'])
    
    # Create lagged ppg column
    team_df['team_topg_lag'] = team_df['team_topg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_topg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_topg_lag'], np.nan)
    team_df['und_topg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_topg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_topg', 'und_topg',
                    'fav_topg_lag', 'und_topg_lag']]

def all_teams_topg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_topg_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_topg_lag': 'first',
        'und_topg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [10]:
# Next we will lag the YPG

def process_ypg_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_ypg'], team_df['und_ypg'])
    team_df['team_ypg'] = np.where(team_df['favorite'] == team_name, team_df['fav_ypg'], team_df['und_ypg'])
    
    # Create lagged ppg column
    team_df['team_ypg_lag'] = team_df['team_ypg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_ypg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_ypg_lag'], np.nan)
    team_df['und_ypg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_ypg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_ypg', 'und_ypg',
                    'fav_ypg_lag', 'und_ypg_lag']]

def all_teams_ypg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_ypg_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_ypg_lag': 'first',
        'und_ypg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [11]:
# Next we will lag the YAPG

def process_yapg_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_yapg'], team_df['und_yapg'])
    team_df['team_yapg'] = np.where(team_df['favorite'] == team_name, team_df['fav_yapg'], team_df['und_yapg'])
    
    # Create lagged ppg column
    team_df['team_yapg_lag'] = team_df['team_yapg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_yapg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_yapg_lag'], np.nan)
    team_df['und_yapg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_yapg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_yapg', 'und_yapg',
                    'fav_yapg_lag', 'und_yapg_lag']]

def all_teams_yapg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_yapg_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_yapg_lag': 'first',
        'und_yapg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [12]:
# Next we will lag the TOFPG

def process_tofpg_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_tofpg'], team_df['und_tofpg'])
    team_df['team_tofpg'] = np.where(team_df['favorite'] == team_name, team_df['fav_tofpg'], team_df['und_tofpg'])
    
    # Create lagged ppg column
    team_df['team_tofpg_lag'] = team_df['team_tofpg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_tofpg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_tofpg_lag'], np.nan)
    team_df['und_tofpg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_tofpg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_tofpg', 'und_tofpg',
                    'fav_tofpg_lag', 'und_tofpg_lag']]

def all_teams_tofpg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_tofpg_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_tofpg_lag': 'first',
        'und_tofpg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [13]:
# Next we will lag the PAPG

def process_papg_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_papg'], team_df['und_papg'])
    team_df['team_papg'] = np.where(team_df['favorite'] == team_name, team_df['fav_papg'], team_df['und_papg'])
    
    # Create lagged ppg column
    team_df['team_papg_lag'] = team_df['team_papg'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_papg_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_papg_lag'], np.nan)
    team_df['und_papg_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_papg_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_papg', 'und_papg',
                    'fav_papg_lag', 'und_papg_lag']]

def all_teams_papg_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_papg_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_papg_lag': 'first',
        'und_papg_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [14]:
# Next we will lag the win_pct

def process_win_pct_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_win_pct'], team_df['und_win_pct'])
    team_df['team_win_pct'] = np.where(team_df['favorite'] == team_name, team_df['fav_win_pct'], team_df['und_win_pct'])
    
    # Create lagged ppg column
    team_df['team_win_pct_lag'] = team_df['team_win_pct'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_win_pct_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_win_pct_lag'], np.nan)
    team_df['und_win_pct_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_win_pct_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_win_pct', 'und_win_pct',
                    'fav_win_pct_lag', 'und_win_pct_lag']]

def all_teams_win_pct_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_win_pct_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_win_pct_lag': 'first',
        'und_win_pct_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [15]:
# Next we will lag the AVG_MOV

def process_avg_mov_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_avg_mov'], team_df['und_avg_mov'])
    team_df['team_avg_mov'] = np.where(team_df['favorite'] == team_name, team_df['fav_avg_mov'], team_df['und_avg_mov'])
    
    # Create lagged ppg column
    team_df['team_avg_mov_lag'] = team_df['team_avg_mov'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_avg_mov_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_avg_mov_lag'], np.nan)
    team_df['und_avg_mov_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_avg_mov_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_avg_mov', 'und_avg_mov',
                    'fav_avg_mov_lag', 'und_avg_mov_lag']]

def all_teams_avg_mov_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_avg_mov_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_avg_mov_lag': 'first',
        'und_avg_mov_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [16]:
# Next we will lag the last_5_win_pct

def process_last_5_win_pct_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_last_5_win_pct'], team_df['und_last_5_win_pct'])
    team_df['team_last_5_win_pct'] = np.where(team_df['favorite'] == team_name, team_df['fav_last_5_win_pct'], team_df['und_last_5_win_pct'])
    
    # Create lagged ppg column
    team_df['team_last_5_win_pct_lag'] = team_df['team_last_5_win_pct'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_last_5_win_pct_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_last_5_win_pct_lag'], np.nan)
    team_df['und_last_5_win_pct_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_last_5_win_pct_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_last_5_win_pct', 'und_last_5_win_pct',
                    'fav_last_5_win_pct_lag', 'und_last_5_win_pct_lag']]

def all_teams_last_5_win_pct_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_last_5_win_pct_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_last_5_win_pct_lag': 'first',
        'und_last_5_win_pct_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [17]:
# Next we will lag the home_win_pct

def process_home_win_pct_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_home_win_pct'], team_df['und_home_win_pct'])
    team_df['team_home_win_pct'] = np.where(team_df['favorite'] == team_name, team_df['fav_home_win_pct'], team_df['und_home_win_pct'])
    
    # Create lagged ppg column
    team_df['team_home_win_pct_lag'] = team_df['team_home_win_pct'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_home_win_pct_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_home_win_pct_lag'], np.nan)
    team_df['und_home_win_pct_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_home_win_pct_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_home_win_pct', 'und_home_win_pct',
                    'fav_home_win_pct_lag', 'und_home_win_pct_lag']]

def all_teams_home_win_pct_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_home_win_pct_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_home_win_pct_lag': 'first',
        'und_home_win_pct_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [18]:
# Next we will lag the away_win_pct

def process_away_win_pct_lag(df, team_name):
    # Create a DataFrame for the team, including both favorite and underdog games
    team_df = df[(df['favorite'] == team_name) | (df['underdog'] == team_name)].copy()
    
    # Sort by season and week
    team_df = team_df.sort_values(['season', 'week'])
    
    # Create a column for the team's score and ppg
    team_df['team_yards'] = np.where(team_df['favorite'] == team_name, team_df['fav_away_win_pct'], team_df['und_away_win_pct'])
    team_df['team_away_win_pct'] = np.where(team_df['favorite'] == team_name, team_df['fav_away_win_pct'], team_df['und_away_win_pct'])
    
    # Create lagged ppg column
    team_df['team_away_win_pct_lag'] = team_df['team_away_win_pct'].shift(1)
    
    # Create columns to merge back to the main DataFrame
    team_df['fav_away_win_pct_lag'] = np.where(team_df['favorite'] == team_name, team_df['team_away_win_pct_lag'], np.nan)
    team_df['und_away_win_pct_lag'] = np.where(team_df['underdog'] == team_name, team_df['team_away_win_pct_lag'], np.nan)
    
    return team_df[['favorite', 'underdog', 'week', 'season', 'fav_away_win_pct', 'und_away_win_pct',
                    'fav_away_win_pct_lag', 'und_away_win_pct_lag']]

def all_teams_away_win_pct_merge(df):
    # Get unique team names
    teams = pd.unique(df[['favorite', 'underdog']].values.ravel('K'))
    
    # Process each team
    all_teams_df = pd.concat([process_away_win_pct_lag(df, team) for team in teams])
    
    # Group by game identifiers and aggregate
    grouped_stats = all_teams_df.groupby(['favorite', 'underdog', 'week', 'season']).agg({
        'fav_away_win_pct_lag': 'first',
        'und_away_win_pct_lag': 'first'
    }).reset_index()
    
    # Merge the results back to the df
    merged_df = pd.merge(df, grouped_stats, on=['favorite', 'underdog', 'week', 'season'], how='left')
    
    return merged_df


In [19]:
# Merging all of this data together

def merge_all_data(df):
    
    df = all_teams_ppg_merge(df)
    df = all_teams_papg_merge(df)
    df = all_teams_ypg_merge(df)
    df = all_teams_yapg_merge(df)
    df = all_teams_topg_merge(df)
    df = all_teams_tofpg_merge(df)
    df = all_teams_avg_mov_merge(df)
    df = all_teams_win_pct_merge(df)
    df = all_teams_last_5_win_pct_merge(df)
    df = all_teams_home_win_pct_merge(df)
    df = all_teams_away_win_pct_merge(df)
    
    return df

final_df = merge_all_data(filtered_df)

In [20]:
final_df.head()

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_ppg,und_ppg,fav_topg,und_topg,fav_ypg,und_ypg,off_rating_fav,off_rating_und,fav_yapg,und_yapg,fav_tofpg,und_tofpg,fav_papg,und_papg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_mov,und_avg_mov,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct,fav_ppg_lag,und_ppg_lag,fav_papg_lag,und_papg_lag,fav_ypg_lag,und_ypg_lag,fav_yapg_lag,und_yapg_lag,fav_topg_lag,und_topg_lag,fav_tofpg_lag,und_tofpg_lag,fav_avg_mov_lag,und_avg_mov_lag,fav_win_pct_lag,und_win_pct_lag,fav_last_5_win_pct_lag,und_last_5_win_pct_lag,fav_home_win_pct_lag,und_home_win_pct_lag,fav_away_win_pct_lag,und_away_win_pct_lag
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,34.0,20.0,0.0,1.0,369.0,360.0,75.55,66.49,360.0,369.0,1.0,0.0,20.0,34.0,65.29,69.00,0,1.0,0.0,1.0,0.0,14.0,-14.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,25.0,38.0,2.0,0.0,506.0,383.0,72.02,72.33,383.0,506.0,0.0,2.0,38.0,25.0,68.86,69.55,0,0.0,1.0,0.0,1.0,-13.0,13.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,38.0,6.0,1.0,3.0,381.0,306.0,75.81,74.89,306.0,381.0,3.0,1.0,6.0,38.0,73.05,69.66,1,1.0,0.0,1.0,0.0,32.0,-32.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,27.0,17.0,2.0,2.0,404.0,254.0,65.68,68.71,254.0,404.0,2.0,2.0,17.0,27.0,64.69,72.58,1,1.0,0.0,1.0,0.0,10.0,-10.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,34.0,30.0,0.0,0.0,372.0,388.0,72.92,68.63,388.0,372.0,0.0,0.0,30.0,34.0,66.19,71.37,0,1.0,0.0,1.0,0.0,4.0,-4.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Checking

In [21]:
n = final_df.isna().sum()

n = n[n > 0]

n

fav_ppg_lag               16
und_ppg_lag               16
fav_papg_lag              16
und_papg_lag              16
fav_ypg_lag               16
und_ypg_lag               16
fav_yapg_lag              16
und_yapg_lag              16
fav_topg_lag              16
und_topg_lag              16
fav_tofpg_lag             16
und_tofpg_lag             16
fav_avg_mov_lag           16
und_avg_mov_lag           16
fav_win_pct_lag           16
und_win_pct_lag           16
fav_last_5_win_pct_lag    16
und_last_5_win_pct_lag    16
fav_home_win_pct_lag      16
und_home_win_pct_lag      16
fav_away_win_pct_lag      16
und_away_win_pct_lag      16
dtype: int64

We can see that all columns that are lagged have 16 Nan values. This is fine since week 1 for the 2020 season will not have any data as we lagged each column by a week. 

So now we have lagged all of our features correctly!

# 3.) Data Imputation

Since our data now has Nan values for all lagged features (features we will use in the model) for Week 1 of 2020, I want to conduct some data imputation rather than just losing this data by getting rid of it.

In [28]:
final_df[final_df['fav_ppg_lag'].isna()]

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_ppg,und_ppg,fav_topg,und_topg,fav_ypg,und_ypg,off_rating_fav,off_rating_und,fav_yapg,und_yapg,fav_tofpg,und_tofpg,fav_papg,und_papg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_mov,und_avg_mov,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct,fav_ppg_lag,und_ppg_lag,fav_papg_lag,und_papg_lag,fav_ypg_lag,und_ypg_lag,fav_yapg_lag,und_yapg_lag,fav_topg_lag,und_topg_lag,fav_tofpg_lag,und_tofpg_lag,fav_avg_mov_lag,und_avg_mov_lag,fav_win_pct_lag,und_win_pct_lag,fav_last_5_win_pct_lag,und_last_5_win_pct_lag,fav_home_win_pct_lag,und_home_win_pct_lag,fav_away_win_pct_lag,und_away_win_pct_lag
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,34.0,20.0,0.0,1.0,369.0,360.0,75.55,66.49,360.0,369.0,1.0,0.0,20.0,34.0,65.29,69.00,0,1.0,0.0,1.0,0.0,14.0,-14.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,25.0,38.0,2.0,0.0,506.0,383.0,72.02,72.33,383.0,506.0,0.0,2.0,38.0,25.0,68.86,69.55,0,0.0,1.0,0.0,1.0,-13.0,13.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,38.0,6.0,1.0,3.0,381.0,306.0,75.81,74.89,306.0,381.0,3.0,1.0,6.0,38.0,73.05,69.66,1,1.0,0.0,1.0,0.0,32.0,-32.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,27.0,17.0,2.0,2.0,404.0,254.0,65.68,68.71,254.0,404.0,2.0,2.0,17.0,27.0,64.69,72.58,1,1.0,0.0,1.0,0.0,10.0,-10.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,34.0,30.0,0.0,0.0,372.0,388.0,72.92,68.63,388.0,372.0,0.0,0.0,30.0,34.0,66.19,71.37,0,1.0,0.0,1.0,0.0,4.0,-4.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Detroit Lions,Chicago Bears,1,23,27,-2.5,42.5,0,0.0,1,1,2020,Chicago Bears,Detroit Lions,27.0,23.0,363.0,0.0,426.0,1.0,23.0,27.0,1.0,0.0,426.0,363.0,73.31,64.60,363.0,426.0,0.0,1.0,27.0,23.0,72.40,67.56,1,0.0,1.0,0.0,1.0,-4.0,4.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Indianapolis Colts,Jacksonville Jaguars,0,20,27,-7.0,44.0,0,0.0,1,1,2020,Jacksonville Jaguars,Indianapolis Colts,27.0,20.0,241.0,0.0,445.0,2.0,20.0,27.0,2.0,0.0,445.0,241.0,76.71,64.73,241.0,445.0,0.0,2.0,27.0,20.0,71.57,65.88,1,0.0,1.0,0.0,1.0,-7.0,7.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Minnesota Vikings,Green Bay Packers,1,34,43,-1.0,44.0,0,0.0,1,1,2020,Green Bay Packers,Minnesota Vikings,43.0,34.0,522.0,0.0,382.0,1.0,34.0,43.0,1.0,0.0,382.0,522.0,72.61,75.15,522.0,382.0,0.0,1.0,43.0,34.0,74.55,67.39,1,0.0,1.0,0.0,1.0,-9.0,9.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,New England Patriots,Miami Dolphins,1,21,11,-7.5,41.5,1,1.0,0,1,2020,New England Patriots,Miami Dolphins,21.0,11.0,357.0,1.0,269.0,3.0,21.0,11.0,1.0,3.0,357.0,269.0,71.41,69.58,269.0,357.0,3.0,1.0,11.0,21.0,70.27,67.25,1,1.0,0.0,1.0,0.0,10.0,-10.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Philadelphia Eagles,Wash

In [43]:
def impute_first_week_lagged_2020(df, lagged_columns):
    # Ensure the dataframe is sorted by season and week
    df = df.sort_values(['season', 'week'])

    # Create a mask for the first week of the 2020 season
    first_week_2020_mask = (df['season'] == 2020) & (df['week'] == 1)

    for col in lagged_columns:
        # Determine the corresponding non-lagged column name
        if col.startswith('fav_'):
            original_col = col.replace('_lag', '')
        elif col.startswith('und_'):
            original_col = col.replace('_lag', '')
        else:
            original_col = col.replace('_lag', '')

        # Impute values for the first week of 2020 using non-lagged data
        df.loc[first_week_2020_mask, col] = df.loc[first_week_2020_mask, original_col]

    return df

# Your list of lagged columns
lagged_columns = ['fav_ppg_lag', 'und_ppg_lag',
       'fav_papg_lag', 'und_papg_lag', 'fav_ypg_lag', 'und_ypg_lag',
       'fav_yapg_lag', 'und_yapg_lag', 'fav_topg_lag', 'und_topg_lag',
       'fav_tofpg_lag', 'und_tofpg_lag', 'fav_avg_mov_lag', 'und_avg_mov_lag',
       'fav_win_pct_lag', 'und_win_pct_lag', 'fav_last_5_win_pct_lag',
       'und_last_5_win_pct_lag', 'fav_home_win_pct_lag',
       'und_home_win_pct_lag', 'fav_away_win_pct_lag', 'und_away_win_pct_lag']

# Apply the function
final_df = impute_first_week_lagged_2020(final_df, lagged_columns)

In [44]:
final_df

,favorite,underdog,fav_home,fav_score,und_score,spread,ou_value,fav_ml_result,fav_sp_result,ou_result,week,season,winner,loser,pts_w,pts_l,yds_w,to_w,yds_l,to_l,fav_ppg,und_ppg,fav_topg,und_topg,fav_ypg,und_ypg,off_rating_fav,off_rating_und,fav_yapg,und_yapg,fav_tofpg,und_tofpg,fav_papg,und_papg,def_rating_fav,def_rating_und,div_game,fav_win_pct,und_win_pct,fav_last_5_win_pct,und_last_5_win_pct,fav_avg_mov,und_avg_mov,fav_home_win_pct,fav_away_win_pct,und_home_win_pct,und_away_win_pct,fav_ppg_lag,und_ppg_lag,fav_papg_lag,und_papg_lag,fav_ypg_lag,und_ypg_lag,fav_yapg_lag,und_yapg_lag,fav_topg_lag,und_topg_lag,fav_tofpg_lag,und_tofpg_lag,fav_avg_mov_lag,und_avg_mov_lag,fav_win_pct_lag,und_win_pct_lag,fav_last_5_win_pct_lag,und_last_5_win_pct_lag,fav_home_win_pct_lag,und_home_win_pct_lag,fav_away_win_pct_lag,und_away_win_pct_lag
0,Kansas City Chiefs,Houston Texans,1,34,20,-9.5,53.5,1,1.0,1,1,2020,Kansas City Chiefs,Houston Texans,34.0,20.0,369.0,0.0,360.0,1.0,34.00,20.00,0.00,1.00,369.00,360.00,75.55,66.49,360.00,369.00,1.00,0.00,20.00,34.00,65.29,69.00,0,1.00,0.00,1.0,0.0,14.00,-14.00,1.00,0.00,0.00,0.00,34.00,20.00,20.00,34.00,369.00,360.00,360.00,369.00,0.00,1.00,1.00,0.00,14.00,-14.00,1.00,0.00,1.0,0.0,1.00,0.00,0.00,0.00
1,Atlanta Falcons,Seattle Seahawks,1,25,38,-1.0,49.5,0,0.0,1,1,2020,Seattle Seahawks,Atlanta Falcons,38.0,25.0,383.0,0.0,506.0,2.0,25.00,38.00,2.00,0.00,506.00,383.00,72.02,72.33,383.00,506.00,0.00,2.00,38.00,25.00,68.86,69.55,0,0.00,1.00,0.0,1.0,-13.00,13.00,0.00,0.00,0.00,1.00,25.00,38.00,38.00,25.00,506.00,383.00,383.00,506.00,2.00,0.00,0.00,2.00,-13.00,13.00,0.00,1.00,0.0,1.0,0.00,0.00,0.00,1.00
2,Baltimore Ravens,Cleveland Browns,1,38,6,-7.0,47.0,1,1.0,0,1,2020,Baltimore Ravens,Cleveland Browns,38.0,6.0,381.0,1.0,306.0,3.0,38.00,6.00,1.00,3.00,381.00,306.00,75.81,74.89,306.00,381.00,3.00,1.00,6.00,38.00,73.05,69.66,1,1.00,0.00,1.0,0.0,32.00,-32.00,1.00,0.00,0.00,0.00,38.00,6.00,6.00,38.00,381.00,306.00,306.00,381.00,1.00,3.00,3.00,1.00,32.00,-32.00,1.00,0.00,1.0,0.0,1.00,0.00,0.00,0.00
3,Buffalo Bills,New York Jets,1,27,17,-6.5,39.5,1,1.0,1,1,2020,Buffalo Bills,New York Jets,27.0,17.0,404.0,2.0,254.0,2.0,27.00,17.00,2.00,2.00,404.00,254.00,65.68,68.71,254.00,404.00,2.00,2.00,17.00,27.00,64.69,72.58,1,1.00,0.00,1.0,0.0,10.00,-10.00,1.00,0.00,0.00,0.00,27.00,17.00,17.00,27.00,404.00,254.00,254.00,404.00,2.00,2.00,2.00,2.00,10.00,-10.00,1.00,0.00,1.0,0.0,1.00,0.00,0.00,0.00
4,Las Vegas Raiders,Carolina Panthers,0,34,30,-3.0,48.0,1,1.0,1,1,2020,Las Vegas Raiders,Carolina Panthers,34.0,30.0,372.0,0.0,388.0,0.0,34.00,30.00,0.00,0.00,372.00,388.00,72.92,68.63,388.00,372.00,0.00,0.00,30.00,34.00,66.19,71.37,0,1.00,0.00,1.0,0.0,4.00,-4.00,0.00,1.00,0.00,0.00,34.00,30.00,30.00,34.00,372.00,388.00,388.00,372.00,0.00,0.00,0.00,0.00,4.00,-4.00,1.00,0.00,1.0,0.0,0.00,0.00,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,Los Angeles Chargers,Kansas City Chiefs,1,12,13,-3.5,35.0,0,0.0,0,18,2023,Kansas City Chiefs,Los Angeles Chargers,13.0,12.0,268.0,1.0,353.0,1.0,20.35,21.82,1.24,1.65,329.35,351.29,72.14,75.52,362.94,289.76,1.24,1.00,23.41,17.29,62.80,67.67,1,0.29,0.65,0.0,0.6,-3.33,1.00,0.22,0.38,0.50,0.78,20.88,22.38,24.06,17.62,327.88,356.50,368.88,285.81,1.25,1.69,1.25,1.00,-17.00,4.00,0.31,0.62,0.2,0.4,0.25,0.50,0.38,0.75
1067,Philadelphia Eagles,New York Giants,0,10,27,-5.0,42.5,0,0.0,0,18,2023,New York Giants,Philadelphia Eagles,27.0,10.0,415.0,1.0,299.0,4.0,25.47,15.65,1.65,1.12,354.35,280.00,79.50,71.33,356.12,361.71,1.06,1.82,25.18,23.94,75.67,71.29,1,0.65,0.35,0.2,0.4,-4.33,2.67,0.75,0.56,0.50,0.22,26.44,14.94,25.06,24.81,357.81,271.56,352.44,365.62,1.50,1.12,1.06,1.69,0.33,-9.00,0.69,0.31,0.2,0.4,0.75,0.43,0.62,0.22
1068,San Francisco 49ers,Los Angeles Rams,1,20,21,

In [45]:
n = final_df.isna().sum()

n = n[n > 0]

n

Series([], dtype: int64)

We now have a data frame that is completely cleaned and ready for the last step of the feature creation process. 

In the next step I will creating some aggregate features that might useful for the model by reducing the number of features, multicollinearity, etc.

# Write to CSV

In [46]:
fp = '/Users/epainter/Desktop/bet_model_v2/data/processing/fc_2.csv'

final_df.to_csv(fp, index=False)

print(f"Data saved to {fp}")

Data saved to /Users/epainter/Desktop/bet_model_v2/data/processing/fc_2.csv
